# Sampling of test dataset

In [229]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import argrelextrema, chirp, find_peaks, peak_widths
from utils import smooth

In [230]:
# Load the data

data_screw = pd.read_csv('../test_results/test_loaded.csv',
                   delimiter=' ')

data_screw_1 = pd.read_csv('../test_results/screw_test_1.csv',
                   delimiter=' ')

data_experiment = pd.read_csv('../test_results/test_joint.csv',
                   delimiter=' ')

data_dummy_normal = pd.read_csv('../test_results/dummy_normal.csv',
                   delimiter=' ')

data_dummy_anomaly = pd.read_csv('../test_results/dummy_anomaly.csv',
                   delimiter=' ')

data_normal = data_dummy_normal.iloc[:]
data_anomaly = data_dummy_anomaly.iloc[:]

In [231]:
# NORMAL DATA
# Find local maxima and minima
indicator_sensor_normal = data_normal['target_qd_3'].values

top_threshold_value = 0.5
low_threshold_value = -0.5

top_thresholded_normal = np.where(indicator_sensor_normal > top_threshold_value, indicator_sensor_normal, 0)
low_thresholded_normal = np.where(indicator_sensor_normal < low_threshold_value, indicator_sensor_normal, 0)

local_maxima_normal = argrelextrema(top_thresholded_normal, np.greater)[0]
local_minima_normal = argrelextrema(low_thresholded_normal, np.less)[0]

# Find corresponding samples in data
df_maxima_normal = data_normal.iloc[local_maxima_normal]
df_minima_normal = data_normal.iloc[local_minima_normal]

print(len(df_maxima_normal.index))
print(len(df_minima_normal.index))

224
224


In [232]:
# ANOMALY DATA
# Find local maxima and minima
indicator_sensor_anomaly = data_anomaly['target_qd_3'].values

top_threshold_value = 0.5
low_threshold_value = -0.5

top_thresholded_anomaly = np.where(indicator_sensor_anomaly > top_threshold_value, indicator_sensor_anomaly, 0)
low_thresholded_anomaly = np.where(indicator_sensor_anomaly < low_threshold_value, indicator_sensor_anomaly, 0)

local_maxima_anomaly = argrelextrema(top_thresholded_anomaly, np.greater)[0]
local_minima_anomaly = argrelextrema(low_thresholded_anomaly, np.less)[0]

# Find corresponding samples in data
df_maxima_anomaly = data_anomaly.iloc[local_maxima_anomaly]
df_minima_anomaly = data_anomaly.iloc[local_minima_anomaly]

print(len(df_maxima_anomaly.index))
print(len(df_minima_anomaly.index))

101
101


In [233]:
# # TODO: Daniella will do second derivatives here in order to find the whole samples.
# joint_accelerations = data['target_qdd_3'].values
# smoothed_indicator_sensor = smooth(indicator_sensor, 30)
# #joint_accelerations = np.gradient(indicator_sensor)
#
# min_val = joint_accelerations.min() * 5
# max_val = joint_accelerations.max() * 5
# #joint_accelerations = np.interp(joint_accelerations, (joint_accelerations.min(), joint_accelerations.max()), (min_val, max_val))
#
# t_indicator_sensor = np.vstack((data['timestamp'], indicator_sensor))
# t_joint_accelerations = np.vstack((data['timestamp'], joint_accelerations))
# idx = np.argwhere(np.diff(np.sign(indicator_sensor - joint_accelerations))).flatten()
#
#
# %matplotlib qt
# plt.plot(indicator_sensor,color='red')
# plt.plot(joint_accelerations,'blue')
# #plt.scatter(local_maxima, joint_accelerations[local_maxima])
# #plt.scatter(local_minima, joint_accelerations[local_minima])
# plt.scatter(idx, indicator_sensor[idx])
# plt.plot()
#
# plt.legend({'target_qdd_3', 'target_qd_3'})
# plt.show()
#
# print(joint_accelerations[local_maxima])

In [234]:
# Create time-series samples of normal data
samples_list_normal = []
i = 0
for index, row in df_maxima_normal.iterrows():
    sample = data_normal.iloc[df_minima_normal.iloc[i].name:index]
    samples_list_normal.append(sample)
    i += 1
single_sample = samples_list_normal[2]

In [235]:
# Create time-series samples of anomaly data
samples_list_anomaly = []
i = 0
for index, row in df_maxima_anomaly.iterrows():
    sample = data_anomaly.iloc[df_minima_anomaly.iloc[i].name:index]
    samples_list_anomaly.append(sample)
    i += 1
single_sample = samples_list_anomaly[2]

In [ ]:
# Measure lengths of the normal samples
samples_length_normal = []
for sample in samples_list_normal:
    samples_length_normal.append(len(sample.index))

In [ ]:
# Measure lengths of the anomaly samples
samples_length_anomaly = []
for sample in samples_list_anomaly:
    samples_length_anomaly.append(len(sample.index))

In [ ]:
# Delete the incorrect samples
samples_list_normal = [x for x in samples_list_normal if len(x.index) <= 281]
samples_length_normal = [x for x in samples_length_normal if x <= 281]

In [ ]:
# Delete the incorrect samples
samples_list_anomaly = [x for x in samples_list_anomaly if len(x.index) <= 87]
samples_length_anomaly = [x for x in samples_length_anomaly if x <= 87]

### Finishing analysis

In [ ]:
# Set plotting data
data = data_anomaly
df_maxima = df_maxima_anomaly
df_minima = df_minima_anomaly
samples_list = samples_list_anomaly
samples_length = samples_length_anomaly

# data = data_normal
# df_maxima = df_maxima_normal
# df_minima = df_minima_normal
# samples_list = samples_list_normal
# samples_length = samples_length_normal

In [ ]:
# Get positions of the longest and shortest samples
longest_sample_pos = samples_length.index(max(samples_length))
shortest_sample_pos = samples_length.index(min(samples_length))

samples_length = np.unique(samples_length)

print('Dataset contains {n_samples} samples'.format(
    n_samples=len(samples_list)))

print('Samples have lengths of {samples_length}'.format(
    samples_length=samples_length
))

longest_sample = samples_list[longest_sample_pos]
shortest_sample = samples_list[shortest_sample_pos]

length_comparison = [shortest_sample, longest_sample]

### Plot the selected data and shade all samples

In [ ]:
# %matplotlib qt
# #plt.plot(data['timestamp'], data['target_q_0'],
# #         label='Line 0')
#
# #plt.plot(data['timestamp'], data['target_q_1'],
# #        label='Line 1')
#
# #plt.plot(data['timestamp'], data['target_q_2'],
# #        label='Line 2')
#
# # Plot the values for joint responsible for movement in axis Z
# plt.plot(data['timestamp'], data['target_q_3'],
#         label='Line 3')
#
# plt.plot(data['timestamp'], data['target_qd_3'],
#         label='Line 4')
#
# # Plot the found local maxima
# for index, row in df_maxima.iterrows():
#     plt.axvline(x=row['timestamp'],
#                 color='green')
#
# # Plot the found local minima
# for index, row in df_minima.iterrows():
#     plt.axvline(x=row['timestamp'],
#                 color='red')
#
# # Plot the thresholds
# plt.axhline(top_threshold_value,
#            color='green',
#             linestyle='--')
#
# plt.axhline(low_threshold_value,
#            color='red',
#            linestyle='--')
#
# # # Shade the samples
# # for sample in samples_list:
# #     plt.axvspan(sample.head(1)['timestamp'].values[0],
# #                 sample.tail(1)['timestamp'].values[0],
# #                 facecolor='b', alpha=0.1)
#
# # plt.tight_layout()
# # plt.autoscale(enable=True, axis='x')
# # plt.autoscale(enable=True, axis='y')
# # plt.legend()
# # plt.show()
#
# # # Find the peaks
# # peaks, _ = find_peaks(indicator_sensor)
# #
# # results_half = peak_widths(indicator_sensor, peaks, rel_height=0.5)
# # results_full = peak_widths(indicator_sensor, peaks, rel_height=0.7)
# #
# # plt.plot(indicator_sensor)
# #
# # plt.plot(peaks, indicator_sensor[peaks], "x")
# # plt.plot(data['target_q_3'])
# #
# # plt.hlines(*results_half[1:], color="C2")
# #
# # plt.hlines(*results_full[1:], color="C3")
#
# # Shade the samples
# for sample in samples_list:
#     plt.axvspan(sample.head(1)['timestamp'].values[0],
#                 sample.tail(1)['timestamp'].values[0],
#                 facecolor='b', alpha=0.1)
#
# plt.tight_layout()
# plt.autoscale(enable=True, axis='x')
# plt.autoscale(enable=True, axis='y')
# plt.legend()
# plt.show()
#

### Plot comparison between the longest and shortest samples

In [ ]:
# %matplotlib qt
#
# #plt.plot(data['timestamp'], data['target_q_0'],
# #         label='Line 0')
#
# #plt.plot(data['timestamp'], data['target_q_1'],
# #        label='Line 1')
#
# #plt.plot(data['timestamp'], data['target_q_2'],
# #        label='Line 2')
#
# # Plot the values for joint responsible for movement in axis Z
# plt.plot(data['timestamp'], data['target_q_3'],
#         label='Line 3')
#
# plt.plot(data['timestamp'], data['target_qd_3'],
#         label='Line 4')
#
# # Plot the thresholds
# plt.axhline(top_threshold_value,
#            color='green',
#             linestyle='--')
#
# plt.axhline(low_threshold_value,
#            color='red',
#            linestyle='--')
#
# # Shade the longest and shortest samples
# for sample in length_comparison:
#     plt.axvspan(sample.head(1)['timestamp'].values[0],
#                 sample.tail(1)['timestamp'].values[0],
#                 facecolor='b', alpha=0.1)
#
# plt.tight_layout()
# plt.autoscale(enable=True, axis='x')
# plt.autoscale(enable=True, axis='y')
# plt.legend()
# plt.show()

### Mix the dataset and save the data

In [ ]:
# Delete timestamp column and pad samples to be the same shape
# TODO: Very inefficient
for id, sample in enumerate(samples_list_normal):
    sample = sample.drop(['timestamp'], axis=1)
    while len(sample.index) < max(samples_length_normal):
        sample = sample.append(pd.Series(0, index=sample.columns), ignore_index=True)
    samples_list_normal[id] = sample
    samples_list_normal[id]['label'] = 0

In [ ]:
# Delete timestamp column and pad samples to be the same shape
# TODO: Very inefficient
for id, sample in enumerate(samples_list_anomaly):
    sample = sample.drop(['timestamp'], axis=1)
    while len(sample.index) < max(samples_length_normal):
        sample = sample.append(pd.Series(0, index=sample.columns), ignore_index=True)
    samples_list_anomaly[id] = sample
    samples_list_anomaly[id]['label'] = 1

In [ ]:
# Create dictionary of samples
training_dictionary = {}
for id, sample in enumerate(samples_list_normal):
    training_dictionary.update({id: sample})

# Concatenate samples
training_dataset = pd.concat(training_dictionary)
training_dataset.index.names = ['sample', 'event']

# Save samples
training_dataset.to_pickle('../datasets/dummy/training.pkl')

In [ ]:
# Create dictionary of samples
test_dictionary = training_dictionary
for id, sample in enumerate(samples_list_anomaly):
    dict_id = id + len(samples_list_normal) + 1
    test_dictionary.update({dict_id: sample})

# Concatenate samples
test_dataset = pd.concat(test_dictionary)
test_dataset.index.names = ['sample', 'event']

# Save samples
test_dataset.to_pickle('../datasets/dummy/test.pkl')

In [ ]:
# Load the training dataset and make numpy array out of it
loaded_dataset = pd.read_pickle('../datasets/dummy/training.pkl')

# Extract the labels and create a samples vector out of it
labels = loaded_dataset.iloc[:, loaded_dataset.columns.get_level_values(0) == 'label']
labels = labels.droplevel('event')
labels = labels[~labels.index.duplicated(keep='first')]
labels_np = np.squeeze(labels.values)

# Drop the labels from data
loaded_dataset = loaded_dataset.drop('label', axis=1)

dim_0 = len(loaded_dataset.index.get_level_values(0).unique())
dim_1 = int(len(loaded_dataset.index.get_level_values(1)) / dim_0)
dim_2 = loaded_dataset.shape[1]

loaded_dataset_np = loaded_dataset.values.reshape((dim_0, dim_1, dim_2))

In [236]:
# Load the training dataset and make numpy array out of it
loaded_dataset = pd.read_pickle('../datasets/dummy/training.pkl')

# Extract the labels and create a samples vector out of it
labels = loaded_dataset.iloc[:, loaded_dataset.columns.get_level_values(0) == 'label']
labels = labels.droplevel('event')
labels = labels[~labels.index.duplicated(keep='first')]
labels_np = np.squeeze(labels.values)

# Drop the labels from data
loaded_dataset = loaded_dataset.drop('label', axis=1)

dim_0 = len(loaded_dataset.index.get_level_values(0).unique())
dim_1 = int(len(loaded_dataset.index.get_level_values(1)) / dim_0)
dim_2 = loaded_dataset.shape[1]

loaded_dataset_np = loaded_dataset.values.reshape((dim_0, dim_1, dim_2))

In [237]:
# Measure lengths of the anomaly samples
samples_length_anomaly = []
for sample in samples_list_anomaly:
    samples_length_anomaly.append(len(sample.index))

In [238]:
# Delete the incorrect samples
samples_list_normal = [x for x in samples_list_normal if len(x.index) <= 281]
samples_length_normal = [x for x in samples_length_normal if x <= 281]

In [239]:
# Delete the incorrect samples
samples_list_anomaly = [x for x in samples_list_anomaly if len(x.index) <= 87]
samples_length_anomaly = [x for x in samples_length_anomaly if x <= 87]

### Finishing analysis

In [240]:
# Set plotting data
data = data_anomaly
df_maxima = df_maxima_anomaly
df_minima = df_minima_anomaly
samples_list = samples_list_anomaly
samples_length = samples_length_anomaly

# data = data_normal
# df_maxima = df_maxima_normal
# df_minima = df_minima_normal
# samples_list = samples_list_normal
# samples_length = samples_length_normal

In [241]:
# Get positions of the longest and shortest samples
longest_sample_pos = samples_length.index(max(samples_length))
shortest_sample_pos = samples_length.index(min(samples_length))

samples_length = np.unique(samples_length)

print('Dataset contains {n_samples} samples'.format(
    n_samples=len(samples_list)))

print('Samples have lengths of {samples_length}'.format(
    samples_length=samples_length
))

longest_sample = samples_list[longest_sample_pos]
shortest_sample = samples_list[shortest_sample_pos]

length_comparison = [shortest_sample, longest_sample]

Dataset contains 76 samples
Samples have lengths of [79 80 85 86 87]


### Plot the selected data and shade all samples

In [242]:
# %matplotlib qt
# #plt.plot(data['timestamp'], data['target_q_0'],
# #         label='Line 0')
#
# #plt.plot(data['timestamp'], data['target_q_1'],
# #        label='Line 1')
#
# #plt.plot(data['timestamp'], data['target_q_2'],
# #        label='Line 2')
#
# # Plot the values for joint responsible for movement in axis Z
# plt.plot(data['timestamp'], data['target_q_3'],
#         label='Line 3')
#
# plt.plot(data['timestamp'], data['target_qd_3'],
#         label='Line 4')
#
# # Plot the found local maxima
# for index, row in df_maxima.iterrows():
#     plt.axvline(x=row['timestamp'],
#                 color='green')
#
# # Plot the found local minima
# for index, row in df_minima.iterrows():
#     plt.axvline(x=row['timestamp'],
#                 color='red')
#
# # Plot the thresholds
# plt.axhline(top_threshold_value,
#            color='green',
#             linestyle='--')
#
# plt.axhline(low_threshold_value,
#            color='red',
#            linestyle='--')
#
# # # Shade the samples
# # for sample in samples_list:
# #     plt.axvspan(sample.head(1)['timestamp'].values[0],
# #                 sample.tail(1)['timestamp'].values[0],
# #                 facecolor='b', alpha=0.1)
#
# # plt.tight_layout()
# # plt.autoscale(enable=True, axis='x')
# # plt.autoscale(enable=True, axis='y')
# # plt.legend()
# # plt.show()
#
# # # Find the peaks
# # peaks, _ = find_peaks(indicator_sensor)
# #
# # results_half = peak_widths(indicator_sensor, peaks, rel_height=0.5)
# # results_full = peak_widths(indicator_sensor, peaks, rel_height=0.7)
# #
# # plt.plot(indicator_sensor)
# #
# # plt.plot(peaks, indicator_sensor[peaks], "x")
# # plt.plot(data['target_q_3'])
# #
# # plt.hlines(*results_half[1:], color="C2")
# #
# # plt.hlines(*results_full[1:], color="C3")
#
# # Shade the samples
# for sample in samples_list:
#     plt.axvspan(sample.head(1)['timestamp'].values[0],
#                 sample.tail(1)['timestamp'].values[0],
#                 facecolor='b', alpha=0.1)
#
# plt.tight_layout()
# plt.autoscale(enable=True, axis='x')
# plt.autoscale(enable=True, axis='y')
# plt.legend()
# plt.show()
#

### Plot comparison between the longest and shortest samples

In [243]:
# %matplotlib qt
#
# #plt.plot(data['timestamp'], data['target_q_0'],
# #         label='Line 0')
#
# #plt.plot(data['timestamp'], data['target_q_1'],
# #        label='Line 1')
#
# #plt.plot(data['timestamp'], data['target_q_2'],
# #        label='Line 2')
#
# # Plot the values for joint responsible for movement in axis Z
# plt.plot(data['timestamp'], data['target_q_3'],
#         label='Line 3')
#
# plt.plot(data['timestamp'], data['target_qd_3'],
#         label='Line 4')
#
# # Plot the thresholds
# plt.axhline(top_threshold_value,
#            color='green',
#             linestyle='--')
#
# plt.axhline(low_threshold_value,
#            color='red',
#            linestyle='--')
#
# # Shade the longest and shortest samples
# for sample in length_comparison:
#     plt.axvspan(sample.head(1)['timestamp'].values[0],
#                 sample.tail(1)['timestamp'].values[0],
#                 facecolor='b', alpha=0.1)
#
# plt.tight_layout()
# plt.autoscale(enable=True, axis='x')
# plt.autoscale(enable=True, axis='y')
# plt.legend()
# plt.show()

### Mix the dataset and save the data

In [244]:
# Delete timestamp column and pad samples to be the same shape
# TODO: Very inefficient
for id, sample in enumerate(samples_list_normal):
    sample = sample.drop(['timestamp'], axis=1)
    while len(sample.index) < max(samples_length_normal):
        sample = sample.append(pd.Series(0, index=sample.columns), ignore_index=True)
    samples_list_normal[id] = sample
    samples_list_normal[id]['label'] = 0

In [245]:
# Delete timestamp column and pad samples to be the same shape
# TODO: Very inefficient
for id, sample in enumerate(samples_list_anomaly):
    sample = sample.drop(['timestamp'], axis=1)
    while len(sample.index) < max(samples_length_normal):
        sample = sample.append(pd.Series(0, index=sample.columns), ignore_index=True)
    samples_list_anomaly[id] = sample
    samples_list_anomaly[id]['label'] = 1

In [301]:
# Create dictionary of samples
training_dictionary = {}
for id, sample in enumerate(samples_list_normal):
    training_dictionary.update({id: sample})

# Concatenate samples
training_dataset = pd.concat(training_dictionary)
training_dataset.index.names = ['sample', 'event']

# Save samples
training_dataset.to_pickle('../datasets/dummy/training.pkl')

In [302]:
# Create dictionary of samples
test_dictionary = training_dictionary
for id, sample in enumerate(samples_list_anomaly):
    dict_id = id + len(samples_list_normal) + 1
    test_dictionary.update({dict_id: sample})

# Concatenate samples
test_dataset = pd.concat(test_dictionary)
test_dataset.index.names = ['sample', 'event']

# Save samples
test_dataset.to_pickle('../datasets/dummy/test.pkl')

In [331]:
# Load the training dataset and make numpy array out of it
loaded_dataset = pd.read_pickle('../datasets/dummy/training.pkl')

# Extract the labels and create a samples vector out of it
labels = loaded_dataset.iloc[:, loaded_dataset.columns.get_level_values(0) == 'label']
labels = labels.droplevel('event')
labels = labels[~labels.index.duplicated(keep='first')]
labels_np = np.squeeze(labels.values)

# Drop the labels from data
loaded_dataset = loaded_dataset.drop('label', axis=1)

dim_0 = len(loaded_dataset.index.get_level_values(0).unique())
dim_1 = int(len(loaded_dataset.index.get_level_values(1)) / dim_0)
dim_2 = loaded_dataset.shape[1]

loaded_dataset_np = loaded_dataset.values.reshape((dim_0, dim_1, dim_2))